In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# Load the dataset
df = pd.read_csv(r"C:\Users\amcys\Documents\02.Education\01.Online Education\03.DataTech Florida - Jerry Strazzeri\09.Trabalhos Finais\Walmart Project\Data\Final_database_V1.csv")
pd.set_option('display.max_columns', None)
df.head(20)

,date,order_id,order_amount,region,items_delivered,items_missing,delivery_hour,customer_id,customer_name,customer_age,driver_id,driver_name,driver_age,driver_trips,product_name,category,price
0,2023-02-12,b0b0ed0b-6d92-4747-a27d-32fa5b293d48,341.92,Altamonte Springs,19,0,04:02:37,WCID5359,Amanda Frank,48,WDID10634,Kevin Castillo DDS,65,48,NaN,NaN,NaN
1,2023-02-12,052ef301-b6a1-4e08-a6eb-8c92f64c11af,113.50,Clermont,8,0,17:58:42,WCID5000,Dustin Spencer,38,WDID10766,Heather Welch,60,25,NaN,NaN,NaN
2,2023-02-12,1c052a51-448f-4859-a739-7066ac41b8e8,442.30,Apopka,19,0,17:14:01,WCID5040,Amy Harrison,72,WDID10966,Shannon Reyes,43,24,NaN,NaN,NaN
3,2023-02-12,ca4bb404-bb71-40ee-925d-b816c97f4a9c,277.32,Clermont,13,0,03:52:42,WCID5867,Jose Jennings,85,WDID11100,Brett Young,48,39,NaN,NaN,NaN
4,2023-02-12,9cc29314-15ad-4259-9547-7b70177ed6de,462.46,Clermont,15,0,14:01:55,WCID6045,Zachary Campbell,60,WDID09899,Stephen Patton,18,47,NaN,NaN,NaN
5,2023-02-12,2be05da2-6166-425d-b2dd-c1f7f14c1935,224.33,Apopka,19,0,15:20:45,WCID5332,Brian Bender,31,WDID09988,Denise Thompson,18,13,NaN,NaN,NaN
6,2023-02-12,fa838369-0405-45b9-9ed1-183245bef63b,400.11,Kissimmee,19,0,11:40:51,WCID5401,Tommy Harvey,59,WDID10472,Brent Green,35,55,NaN,NaN,NaN
7,2023-02-12,e15d2852-0bd6-43bf-bb57-c0c9382547c0,117.29,Altamonte Springs,15,0,19:28:05,WCID5649,Jonathan Stevens,54,WDID10693,Chad Hill,62,38,NaN,NaN,NaN
8,2023-02-12,6db783fb-047c-4eb6-991b-77c24580e2d6,218.04,Apopka,12,0,01:14:21,WCID5147,Katie Mcdaniel,60,WDID10794,James Shepherd,32,27,NaN,NaN,NaN
9,2023-02-12,47bfda64-1b32-45fe-b19d-f8479d4a71a3,494.00,Kissimmee,13,0,05:03:52,WCID6168,James Woodard,35,WDID10924,Tammy Hunt,61,59,NaN,NaN,NaN


In [ ]:
# Here we already can check a datatype error: date and delivery_hour columns are set as "object" and not time and datetime
# Other mistake is the labeling of the last three rows, they must be clearer

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10161 entries, 0 to 10160
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             10161 non-null  object 
 1   order_id         10161 non-null  object 
 2   order_amount     10161 non-null  float64
 3   region           10161 non-null  object 
 4   items_delivered  10161 non-null  int64  
 5   items_missing    10161 non-null  int64  
 6   delivery_hour    10161 non-null  object 
 7   customer_id      10161 non-null  object 
 8   customer_name    10161 non-null  object 
 9   customer_age     10161 non-null  int64  
 10  driver_id        10161 non-null  object 
 11  driver_name      10161 non-null  object 
 12  driver_age       10161 non-null  int64  
 13  driver_trips     10161 non-null  int64  
 14  product_name     1662 non-null   object 
 15  category         1662 non-null   object 
 16  price            1662 non-null   float64
dtypes: float64(2

In [14]:
# Another suspect thing happening is that for some registers, the value lost is bigger than the order price
# We can assume that the "order_amount" is the value without the lost product(s) prices
# This is a data we got in the end of the year, the person probably got time to have his refund. That can be a cause
# We'll create an extra column with the real_order_amount

df[df['price'] > 0.1].head(5)

,date,order_id,order_amount,region,items_delivered,items_missing,delivery_hour,customer_id,customer_name,customer_age,driver_id,driver_name,driver_age,driver_trips,product_name,category,price
18,2023-02-13,3ddfcc22-f1ae-4e28-a87e-fef4b36a0770,78.28,Clermont,10,2,05:57:03,WCID5065,Bruce Ruiz,48,WDID10064,Charlotte Davila,18,37,Hummus,Pantry,7.92
19,2023-02-13,3ddfcc22-f1ae-4e28-a87e-fef4b36a0770,78.28,Clermont,10,2,05:57:03,WCID5065,Bruce Ruiz,48,WDID10064,Charlotte Davila,18,37,Olive Oil,Pantry,6.09
20,2023-02-13,de6343f9-fa09-4cac-8da8-7c2706825e22,133.73,Sanford,13,2,11:20:24,WCID5028,Allison Barker,88,WDID10389,Megan Smith,24,64,Croutons,Pantry,13.31
21,2023-02-13,de6343f9-fa09-4cac-8da8-7c2706825e22,133.73,Sanford,13,2,11:20:24,WCID5028,Allison Barker,88,WDID10389,Megan Smith,24,64,Ground Beef,Pantry,13.27
22,2023-02-13,fe4bffcc-e466-4f05-b111-777a4808abcd,980.99,Sanford,15,1,12:10:18,WCID5145,Joshua Davis,34,WDID10066,Ryan Gomez,18,60,Flour,Pantry,3.48


In [ ]:
# Correct date column type 

df['date'] = pd.to_datetime(df['date'])
df['date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 10161 entries, 0 to 10160
Series name: date
Non-Null Count  Dtype         
--------------  -----         
10161 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 79.5 KB


pandas._libs.tslibs.timestamps.Timestamp

In [19]:
# Correct delivery_hour to time

df['delivery_hour'] = pd.to_datetime(df['delivery_hour'], format='%H:%M:%S', errors='coerce').dt.time
type(df['delivery_hour'].iloc[0])

datetime.time

In [58]:
# Rename Last Three Columns

df = df.rename(columns={'product_name': 'missing_product_name', 
                   'category': 'missing_product_category',
                   'price': 'missing_amount'})
df.head()

,date,order_id,order_amount,region,items_delivered,items_missing,delivery_hour,customer_id,customer_name,customer_age,driver_id,driver_name,driver_age,driver_trips,missing_product_name,missing_product_category,missing_amount
0,2023-02-12,b0b0ed0b-6d92-4747-a27d-32fa5b293d48,341.92,Altamonte Springs,19,0,04:02:37,WCID5359,Amanda Frank,48,WDID10634,Kevin Castillo DDS,65,48,NaN,NaN,NaN
1,2023-02-12,052ef301-b6a1-4e08-a6eb-8c92f64c11af,113.50,Clermont,8,0,17:58:42,WCID5000,Dustin Spencer,38,WDID10766,Heather Welch,60,25,NaN,NaN,NaN
2,2023-02-12,1c052a51-448f-4859-a739-7066ac41b8e8,442.30,Apopka,19,0,17:14:01,WCID5040,Amy Harrison,72,WDID10966,Shannon Reyes,43,24,NaN,NaN,NaN
3,2023-02-12,ca4bb404-bb71-40ee-925d-b816c97f4a9c,277.32,Clermont,13,0,03:52:42,WCID5867,Jose Jennings,85,WDID11100,Brett Young,48,39,NaN,NaN,NaN
4,2023-02-12,9cc29314-15ad-4259-9547-7b70177ed6de,462.46,Clermont,15,0,14:01:55,WCID6045,Zachary Campbell,60,WDID09899,Stephen Patton,18,47,NaN,NaN,NaN


In [ ]:
# Create New dataset with only orders with lost products
df_lost = df[df['missing_amount'] > 0]
df_lost.describe()

,date,order_amount,items_delivered,items_missing,customer_age,driver_age,driver_trips,missing_amount
count,1662,1662.00000,1662.000000,1662.000000,1662.000000,1662.000000,1662.000000,1662.000000
mean,2023-07-01 06:44:37.256317696,342.90855,10.069795,1.204573,54.422984,35.383875,44.950060,89.875217
min,2023-01-01 00:00:00,20.54000,1.000000,1.000000,18.000000,18.000000,11.000000,2.160000
25%,2023-04-05 00:00:00,159.44000,5.000000,1.000000,36.000000,20.000000,29.000000,8.210000
50%,2023-07-04 00:00:00,304.60000,10.000000,1.000000,56.000000,32.000000,45.000000,12.530000
75%,2023-09-28 00:00:00,446.51000,15.000000,1.000000,73.000000,50.000000,62.000000,18.350000
max,2023-12-31 00:00:00,1354.71000,19.000000,3.000000,90.000000,65.000000,78.000000,908.870000
std,NaN,253.47969,5.518722,0.454057,21.108584,15.580617,18.958951,201.186840


In [83]:
# Create a new small database
df_lost_cost_driv = df_lost[['order_id', 'customer_name', 'driver_name', 'driver_trips']]

# Remove duplicates
df_lost_cost_driv = df_lost_cost_driv.drop_duplicates()

# Create variables
suspect_customers = df_lost_cost_driv['customer_name'].value_counts()
suspect_drivers = df_lost_cost_driv['driver_name'].value_counts()

df_lost_cost_driv['customer_incidence'] = df_lost_cost_driv['customer_name'].map(suspect_customers)
df_lost_cost_driv['driver_incidence'] = df_lost_cost_driv['driver_name'].map(suspect_drivers)

# Chances of being a fraudlent customer
# 1 <= incidence < 2 -> low
# 2 <= incidence < 3 -> Medium
# 3 <= incidence < 5 -> High
# 5 <= incidence -> Very High 

# Chances of being a fraudlent driver
# fraud_driver% = df_lost_cost_driv['driver_incidence'] / df_lost_cost_driv['driver_trips']
# 0.00 <= incidence < 0.1 -> low
# 0.1 <= incidence < 0.15 -> Medium
# 0.15 <= incidence < 0.25 -> High
# 0.25 <= incidence -> Very High

def customer_fraud_rating(x):
    if x >= 1 and x < 2:
        return 'Low'
    elif x >= 2 and x < 3:
        return 'Medium'
    elif x >=3 and x < 5:
        return 'High'
    else:
       return 'Very High'

df_lost_cost_driv['perc_lost_products'] = round(df_lost_cost_driv['driver_incidence'] / df_lost_cost_driv['driver_trips'], 3)

def driver_fraud_rating(x):
    if x >= 0.0 and x < 0.1:
        return 'Low'
    elif x >= 0.1 and x < 0.15:
        return 'Medium'
    elif x >=0.15 and x < 0.2:
        return 'High'
    else:
       return 'Very High'
    
df_lost_cost_driv['customer_fraud_rating'] = df_lost_cost_driv['customer_incidence'].apply(customer_fraud_rating)
df_lost_cost_driv['driver_fraud_rating'] = df_lost_cost_driv['perc_lost_products'].apply(driver_fraud_rating)
df_lost_cost_driv.head(25)







,order_id,customer_name,driver_name,driver_trips,customer_incidence,driver_incidence,perc_lost_products,customer_fraud_rating,driver_fraud_rating
18,3ddfcc22-f1ae-4e28-a87e-fef4b36a0770,Bruce Ruiz,Charlotte Davila,37,2,4,0.108,Medium,Medium
20,de6343f9-fa09-4cac-8da8-7c2706825e22,Allison Barker,Megan Smith,64,2,4,0.062,Medium,Low
22,fe4bffcc-e466-4f05-b111-777a4808abcd,Joshua Davis,Ryan Gomez,60,2,4,0.067,Medium,Low
23,849e79f0-13bf-4a80-bc02-62c208ebe4cd,Matthew Osborne,Kevin Gilmore,11,3,3,0.273,High,Very High
24,9923a278-e245-4836-aa35-5f6b7f28428f,Mr. Gregory Luna,Ronald Doyle,78,4,3,0.038,High,Low
25,227670af-aeb6-4301-8030-c9960d6b8b90,Jonathan Rivera,Leslie Powell,47,1,4,0.085,Low,Low
50,502f8720-8f0b-42dd-b65c-53bf089dcefd,Benjamin Harvey,Sharon Hanson,15,1,3,0.200,Low,Very High
51,d9d0baa7-b40d-4248-bfd8-d1d33eeecc06,April Hall,Teresa Trujillo,50,2,4,0.080,Medium,Low
52,d7fbb39b-0e73-4523-b0a1-cce7af949ea0,Caitlyn Velasquez,Jennifer Medina,47,2,3,0.064,Medium,Low
73,d105773f-d566-4aa2-87b2-c5245539e6f5,Amanda Brown,Caleb Martinez,57,2,4,0.070,Medium,Low
